# Formato provisional --> anterior

In [1]:
CPG = 'yogurt'
FILE_NAME = f'prov_{CPG}.csv'

## Importar

In [2]:
# %load basic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',200)
pd.options.display.float_format = '{:.2f}'.format
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Catalogo/data/'
sorted(os.listdir(BASE_DIR))

['.DS_Store',
 'nielsen_coca.csv',
 'nielsen_coca_v2.csv',
 'nielsen_danone.csv',
 'nielsen_essity.csv',
 'nielsen_rb.csv',
 'pbbc.csv',
 'perf_prod_2005_2103.csv',
 'pre_nielsen_sigma.csv',
 'products_coca.csv',
 'products_coca_correct.csv',
 'products_danone.csv',
 'products_danone_may20_mar21.csv',
 'products_essity.csv',
 'products_essity_correct.csv',
 'products_rb.csv',
 'products_rb_correct.csv',
 'products_sigma.csv',
 'products_sigma_correct.csv',
 'prov_coca.csv',
 'prov_essity.csv',
 'prov_rb.csv',
 'prov_sigma.csv',
 'prov_yogurt.csv',
 'som_coca.csv',
 'som_danone.csv',
 'som_essity.csv',
 'som_rb.csv']

In [3]:
import chardet
with open(os.path.join(BASE_DIR,FILE_NAME), 'rb') as rawdata:
    encod = chardet.detect(rawdata.read(1000))['encoding']
print(encod)

UTF-16


In [4]:
df = pd.read_csv(os.path.join(BASE_DIR,FILE_NAME),encoding=encod,sep='\t')
df = df[df['month']!='Total general'].copy()
for col in df.iloc[:,8:].columns:
    df[col] = df[col].str.replace(',','').str.replace('%','').astype(int,errors='ignore')
    
backup = df.copy()
df.sample()

,month,store,category id,product name,product id,barcodes,package,brand,sales pop,sales non pop,orders pop,orders non pop,units found pop,units found non pop,units requested pop,units requested non pop,found rate pop,found rate non pop
1678,01/03/2021,Fresko,24,Yoghurt con manzana,2065145,{7501020563818},120 g,Lala,0,49,0,2,0,10,0,10,NaN,100.0


In [5]:
import re

cat = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv')).astype(str).drop_duplicates('product_id')
df = df.merge(cat[['category_name','product_id']],how='left',
              left_on='product id',
              right_on='product_id').iloc[:,:-1]
df.sample()

,month,store,category id,product name,product id,barcodes,package,brand,sales pop,sales non pop,orders pop,orders non pop,units found pop,units found non pop,units requested pop,units requested non pop,found rate pop,found rate non pop,category_name
5193,01/04/2021,HEB,24,Yoghurt batido estilo griego sabor natural mor...,548998,{75069247},120 g,Lala,1052,1136,41,42,106,115,119,120,100.0,100.0,Yoghurt


## Funciones

In [6]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

## Transformar

In [7]:
df = df[['category_name']+list(df.columns)[:-1]].copy()
orden = {'month': 'Mes de date',
         'store': 'store',
         'category_name':'category',
         'category id':'category_id',
         'product name': 'product name',
         'product id': 'product ID',
         'barcodes': 'product barcodes',
         'package': 'product package',
         'brand': 'brand name'}
df.rename(columns=orden,inplace=True)
df.sample()

,category,Mes de date,store,category_id,product name,product ID,product barcodes,product package,brand name,sales pop,sales non pop,orders pop,orders non pop,units found pop,units found non pop,units requested pop,units requested non pop,found rate pop,found rate non pop
3126,Yoghurt,01/03/2021,Sumesa,24,Yoghurt bebible con guayaba de michoacán,1153154,{7506443101500},240 g,Danone,0,29,0,1,0,2,0,2,NaN,100.0


In [8]:
df = df.iloc[:,:9].join(df.filter(like='pop').melt(ignore_index=False))
df['value'] = df['value'].astype(float)
df['test'] = df['variable'].apply(lambda x: re.split('(non|pop)',x,maxsplit=1))
df['metric'] = [x[0] for x in df['test']]
df['audience'] = [(x[1]+x[2]).strip() for x in df['test']]
df.sample()

,category,Mes de date,store,category_id,product name,product ID,product barcodes,product package,brand name,variable,value,test,metric,audience
5353,Yoghurt,01/04/2021,HEB,24,Yoghurt natural sin azúcar,368938,{7501032398439},900 g,Danone,found rate pop,100.00,"[found rate , pop, ]",found rate,pop


In [9]:
df = df.fillna('').pivot_table(index=list(df.iloc[:,:9].columns)+['audience'],
                               columns='metric',values='value',aggfunc='sum').reset_index()
df.columns = [x.strip() for x in df.columns]
df.insert(12,'users','')
df.insert(13,'avg. ticket','')
df['avg. ticket'] = (df['sales']+1) / (df['orders']+1)
df.sample()

,category,Mes de date,store,category_id,product name,product ID,product barcodes,product package,brand name,audience,found rate,orders,users,avg. ticket,sales,units found,units requested
4900,Yoghurt,01/03/2021,HEB,24,Yoghurt triple cero manzana verde,313846,{7501032397548},125 g,Vitalínea,non pop,100.0,19.0,,35.55,710.0,116.0,116.0


In [10]:
df = df[['Mes de date', 'store', 'category', 'category_id', 'product ID',
         'product name', 'product barcodes', 'product package', 'brand name',
         'audience', 'orders', 'sales', 'users', 'avg. ticket', 'units found',
         'units requested', 'found rate']].copy()
meses = dict(zip(range(1,13),['enero','febrero','marzo','abril','mayo','junio','julio',
                              'agosto','septiembre','octubre','noviembre','diciembre']))
df['Mes de date'] = [meses[x.month]+' '+str(x.year) for x in pd.to_datetime(df['Mes de date'],
                                                                            dayfirst=True)]
df.sample(2)

,Mes de date,store,category,category_id,product ID,product name,product barcodes,product package,brand name,audience,orders,sales,users,avg. ticket,units found,units requested,found rate
10094,abril 2021,Fresko,Yoghurt,24,3036266,Yoghurt con fresa,{7501055914463},900 g,Alpura,non pop,6.0,271.0,,38.86,8.0,8.0,100.0
990,marzo 2021,Alsuper,Yoghurt,24,369325,Bebida láctea sabor fresa,{7501032398583},220 g,Danup,non pop,5.0,284.0,,47.5,36.0,36.0,100.0


In [11]:
backup[['sales pop','sales non pop']].sum().sum() - df['sales'].sum()

0.0

## Exportar

In [12]:
df.to_csv(os.path.join(BASE_DIR,f'products_{CPG}_correct.csv'),index=False)

## Fin

In [13]:
time_exp(time.time() - start)
tono()

0 minutos con 6.89 segundos
